In [1]:
import os
import torch.nn as nn
import torch.nn.functional as F
import torch
import shutil
import kagglehub
import torchvision.models as models
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import random_split,DataLoader
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
# Download the dataset
path = kagglehub.dataset_download("tawsifurrahman/covid19-radiography-database")
path

'/kaggle/input/covid19-radiography-database'

In [3]:
dataset_folder = os.path.join(path,'COVID-19_Radiography_Dataset')
os.listdir(dataset_folder)

['Lung_Opacity.metadata.xlsx',
 'Normal.metadata.xlsx',
 'README.md.txt',
 'COVID.metadata.xlsx',
 'Normal',
 'Lung_Opacity',
 'Viral Pneumonia.metadata.xlsx',
 'Viral Pneumonia',
 'COVID']

In [4]:
# Define source and destination folder
source_root = os.path.join(path,'COVID-19_Radiography_Dataset')
target_root = '/content/Datasets/COVID19_CLASSIFICATION'
classes = ['COVID', 'Normal', 'Lung_Opacity', 'Viral Pneumonia']
for cls in classes:
  src_folder = os.path.join(source_root,cls,'images')
  dst_folder = os.path.join(target_root,cls)
  os.makedirs(dst_folder,exist_ok=True)

  for file in os.listdir(src_folder):
    src_file = os.path.join(src_folder,file)
    dst_file = os.path.join(dst_folder,file)
    if os.path.isfile(src_file):
      shutil.copy2(src_file,dst_file)





In [5]:
transform = transforms.Compose([
  transforms.Resize((224, 224)),
  transforms.ToTensor(),
])
dataset = datasets.ImageFolder(root='/content/Datasets/COVID19_CLASSIFICATION',transform=transform)

In [6]:
train_size = int(0.8*len(dataset))
val_size = len(dataset)- train_size

# Split train and validation dataset
train_dataset, val_dataset = random_split(dataset,[train_size,val_size])

In [7]:
# Create data loaders
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=32,shuffle=False)
# define device and model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
# Use Resnet as a feature extractor
resnet = models.resnet18(pretrained= True)
# Remove fully connected layer
resnet = nn.Sequential(*list(resnet.children())[:-1])
resnet.eval().to(device)
# Function to extract features
def extract_features(dataloader):
  features = []
  labels = []

  with torch.no_grad():
    for inputs, targets in dataloader:
      inputs,targets = inputs.to(device),targets.to(device)
      outputs = resnet(inputs)
      outputs = outputs.view(outputs.size(0),-1) # Flatten
      features.append(outputs.cpu())
      labels.append(targets.cpu())

  return torch.cat(features),torch.cat(labels)


# Extract train and validation data
X_train, y_train = extract_features(train_loader)
X_val, y_val = extract_features(val_loader)

# Train and evaluate Random Forest model
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred_rf = rf.predict(X_val)
print("Random Forest Accuracy:", accuracy_score(y_val, y_pred_rf))

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 230MB/s]


Random Forest Accuracy: 0.8027403732577368


In [9]:
# Define a custom cnn
class CovidCNN(nn.Module):
  def __init__(self,num_classes):
    super().__init__()
    self.conv1 = nn.Conv2d(3,32, kernel_size=3,padding=1)
    self.conv2 = nn.Conv2d(32,64,kernel_size=3,padding=1)
    self.conv3 = nn.Conv2d(64,128,kernel_size=3,padding=1)

    self.pool = nn.MaxPool2d(2,2)
    self.dropout = nn.Dropout(0.2)

    self.fc1 = nn.Linear(128*28*28,256)
    self.fc2 = nn.Linear(256,num_classes)

  def forward(self,x):
    # x [3,224,224]
    pass
    x = self.pool(F.relu(self.conv1(x)))    #[32,112,112]
    x = self.pool(F.relu(self.conv2(x)))    #[64,56,56]
    x = self.pool(F.relu(self.conv3(x)))    #[128,28,28]

    x = x.view(x.size(0),-1) # Flatten
    x = self.fc1(x)
    x = self.fc2(x)
    return x




In [10]:
# Define model, loss function and optimizer
model = CovidCNN(num_classes=4).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.0001)
num_epochs = 10

In [11]:
# Training and validation custom cnn
print('Custom CNN model\n')
for epoch in range(num_epochs):
  model.train()
  running_loss = 0.0
  correct,total = 0,0

  for inputs,labels in train_loader:
    inputs, labels = inputs.to(device),labels.to(device)
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()

    running_loss+= loss.item()
    _,pred = outputs.max(1)
    correct +=(pred==labels).sum().item()
    total+= labels.size(0)

  train_acc = 100* correct/total

  # validation
  model.eval()
  val_correct = 0
  val_total = 0
  with torch.no_grad():
    for inputs, labels in val_loader:
      inputs, labels = inputs.to(device),labels.to(device)
      outputs = model(inputs)
      _,pred = outputs.max(1)
      val_correct +=(pred==labels).sum().item()
      val_total+= labels.size(0)

  val_acc = 100* val_correct/val_total

  print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {running_loss:.4f} | Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")







Custom CNN model

Epoch 1/10 | Train Loss: 358.6287 | Train Acc: 72.86% | Val Acc: 80.94%
Epoch 2/10 | Train Loss: 246.7323 | Train Acc: 82.62% | Val Acc: 82.99%
Epoch 3/10 | Train Loss: 209.2224 | Train Acc: 85.21% | Val Acc: 84.34%
Epoch 4/10 | Train Loss: 182.0770 | Train Acc: 87.17% | Val Acc: 85.57%
Epoch 5/10 | Train Loss: 160.0156 | Train Acc: 88.63% | Val Acc: 85.57%
Epoch 6/10 | Train Loss: 143.8477 | Train Acc: 89.90% | Val Acc: 86.23%
Epoch 7/10 | Train Loss: 125.1135 | Train Acc: 91.32% | Val Acc: 87.72%
Epoch 8/10 | Train Loss: 113.9557 | Train Acc: 91.94% | Val Acc: 87.34%
Epoch 9/10 | Train Loss: 102.5429 | Train Acc: 92.94% | Val Acc: 87.41%
Epoch 10/10 | Train Loss: 88.8428 | Train Acc: 93.72% | Val Acc: 87.27%


In [12]:
from torchvision import models

# Define model, loss function and optimizer
model_pretrain = models.resnet18(pretrained=True)
model_pretrain.fc = nn.Linear(model_pretrain.fc.in_features,4)
model_pretrain.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_pretrain.parameters(),lr = 0.0001)
num_epochs = 10

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
print('Pretrained Resnet model\n')
# Training and validation custom cnn
for epoch in range(num_epochs):
  model_pretrain.train()
  running_loss = 0.0
  correct,total = 0,0

  for inputs,labels in train_loader:
    inputs, labels = inputs.to(device),labels.to(device)
    optimizer.zero_grad()
    outputs = model_pretrain(inputs)
    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()

    running_loss+= loss.item()
    _,pred = outputs.max(1)
    correct +=(pred==labels).sum().item()
    total+= labels.size(0)

  train_acc = 100* correct/total

  # validation
  model_pretrain.eval()
  val_correct = 0
  val_total = 0
  with torch.no_grad():
    for inputs, labels in val_loader:
      inputs, labels = inputs.to(device),labels.to(device)
      outputs = model_pretrain(inputs)
      _,pred = outputs.max(1)
      val_correct +=(pred==labels).sum().item()
      val_total+= labels.size(0)

  val_acc = 100* val_correct/val_total

  print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {running_loss:.4f} | Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")



Pretrained Resnet model

Epoch 1/10 | Train Loss: 135.1109 | Train Acc: 91.08% | Val Acc: 89.72%
Epoch 2/10 | Train Loss: 69.4646 | Train Acc: 95.45% | Val Acc: 89.20%
Epoch 3/10 | Train Loss: 49.3516 | Train Acc: 96.87% | Val Acc: 93.01%
Epoch 4/10 | Train Loss: 26.5528 | Train Acc: 98.33% | Val Acc: 92.98%
Epoch 5/10 | Train Loss: 28.7806 | Train Acc: 97.92% | Val Acc: 95.39%
Epoch 6/10 | Train Loss: 12.1475 | Train Acc: 99.24% | Val Acc: 94.38%
Epoch 7/10 | Train Loss: 13.6117 | Train Acc: 99.16% | Val Acc: 95.23%
Epoch 8/10 | Train Loss: 18.6330 | Train Acc: 98.88% | Val Acc: 94.85%
Epoch 9/10 | Train Loss: 5.2040 | Train Acc: 99.76% | Val Acc: 95.63%
Epoch 10/10 | Train Loss: 10.7054 | Train Acc: 99.28% | Val Acc: 94.16%
